In [1]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

     |████████████████████████████████| 1.8MB 3.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 150.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 233.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fosforml to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 236.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 254.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 1.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 206.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 250.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 304.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 27.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 50.5 MB/s eta 0:00:000

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 241.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 276.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 248.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 269.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 61.6 MB/s eta 0:00:00a 0:0

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Found existing installation: urllib3 1.26.15
Uninstalling urllib3-1.26.15:
  Successfully uninstalled urllib3-1.26.15
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.0.0 requires cloudpickle==1.6.0, but you have cloudpickle 2.2.1 which is incompatible.
fosforml 1.0.0 requires PyYAML==6.0, but you have pyyaml 6.0.1 which is incompa

In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
from datetime import datetime, timedelta

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns

Matplotlib created a temporary cache directory at /tmp/matplotlib-f765jk7t because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS")
data

In [3]:
data = pd.read_csv('./bookings.csv')
data

,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,City Hotel,1,272,2021,7,29,16,2021-07-16,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,City Hotel,1,258,2021,7,27,2,2021-07-02,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,City Hotel,1,279,2021,7,30,23,2021-07-23,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117100,Resort Hotel,0,269,2023,8,34,24,2023-08-24,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
117101,Resort Hotel,0,212,2023,8,35,31,2023-08-31,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
117102,Resort Hotel,0,204,2023,8,35,29,2023-08-29,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
117103,Resort Hotel,0,161,2023,8,35,31,2023-08-31,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [6]:
data.tail()

,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
117100,Resort Hotel,0,269,2023,8,34,24,2023-08-24,Check.Out,2023-09-10,17,17.0,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
117101,Resort Hotel,0,212,2023,8,35,31,2023-08-31,Check.Out,2023-09-10,10,10.0,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
117102,Resort Hotel,0,204,2023,8,35,29,2023-08-29,Check.Out,2023-09-12,14,14.0,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
117103,Resort Hotel,0,161,2023,8,35,31,2023-08-31,Check.Out,2023-09-14,14,14.0,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06
117104,Resort Hotel,0,211,2023,8,35,31,2023-08-31,Check.Out,2023-09-14,14,14.0,4,10,2,0,0,2,1,14,HB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,112.80


In [5]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')
data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')
data['EXPECTED_ARRIVAL_DATE'] - data['RESERVATION_STATUS_DATE']

<ipython-input-5-48c2fc89ab93>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')


0        272 days
1        307 days
2        258 days
3        307 days
4        279 days
           ...   
117100   -17 days
117101   -10 days
117102   -14 days
117103   -14 days
117104   -14 days
Length: 117105, dtype: timedelta64[ns]

In [ ]:
data['LEAD_TIME'].min()

In [ ]:
data['LEAD_TIME'].max()

In [ ]:
data['LEAD_TIME'].mean()

In [ ]:
data.columns

In [ ]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).min()

In [ ]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).max()

In [ ]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')
data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')

In [ ]:
data.sort_values(by='EXPECTED_ARRIVAL_DATE')['EXPECTED_ARRIVAL_DATE']

In [ ]:
original_last_date = datetime(2023, 8, 31)
target_date = datetime(2024, 6, 30)

In [ ]:
original_min_date = data['EXPECTED_ARRIVAL_DATE'].min()
original_min_date

In [ ]:
days_to_add = (target_date - original_last_date).days
days_to_add

In [ ]:
data['NEW_LEAD_TIME']iloc = 0

In [ ]:
for index, row in data.iterrows():
    data[]data['EXPECTED_ARRIVAL_DATE'] - timedelta(days=data['LEAD_TIME'])

In [ ]:
data['EXPECTED_ARRIVAL_DATE'] = data['EXPECTED_ARRIVAL_DATE'] + timedelta(days=days_to_add)
data['RESERVATION_STATUS_DATE'] = data['EXPECTED_ARRIVAL_DATE'] - 

In [ ]:
data['LEAD_TIME_UPDATED'] = data['EXPECTED_ARRIVAL_DATE'] - data['RESERVATION_STATUS_DATE']

In [ ]:
data['LEAD_TIME_UPDATED'].min()

In [ ]:
data['LEAD_TIME_UPDATED'].max()

In [ ]:
data['LEAD_TIME_UPDATED'] = data['LEAD_TIME_UPDATED'].dt.days

In [ ]:
data['LEAD_TIME_UPDATED']

In [ ]:
data[data["LEAD_TIME_UPDATED"] < 0]

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

In [ ]:
data.info()

In [ ]:
data["EXPECTED_ARRIVAL_DATE"] = pd.to_datetime(data["EXPECTED_ARRIVAL_DATE"]).dt.strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
data.info()

In [ ]:
df_model=session.createDataFrame(
        data_backup.values.tolist(),
        schema=data_backup.columns.tolist())
df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.BOOKINGS")

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].isna().sum()

In [ ]:
data_backup = data.dropna(axis=0)

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
117105 - 117009